In [ ]:
%pylab inline

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [ ]:
from monary import Monary
import numpy as np
import pandas as pd
from monary.monary import mvoid_to_bson_id
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import cluster
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction import DictVectorizer

In [ ]:
mon = Monary("127.0.0.1")

# Capsule dataframe

In [ ]:
columns = [
    "Capsule id",
    #"date created",
    #"date updated",
    "capsule name",
    "type of author",
    "author user id",
    "is draft",
    "is public",
    "approximate capsule duration"

]
arrays = mon.query(
"educanti", # database name
"capsule", # collection name
{}, # query spec
[
        "_id",
        #"date_created",
        #"date_updated",
        "name",
        "author._ref.$ref",
        "author._ref.$id",
        "publication.is_draft",
        "publication.is_public",
        "approximate_duration"

], # field names (in Mongo record)
[
        "id",
    #   "date",
        #"date",
        "string:100",
        "string:20",
        "id",
        "bool",
        "bool",
        "float32"
  
    ] # Monary field types (see below)
)
df_capsule = np.matrix(arrays).transpose() 

df_capsule = pd.DataFrame(df_capsule, columns=columns)
mvoid_to_bson_id(arrays[0][0])

In [ ]:
df_capsule["Capsule id"] = df_capsule["Capsule id"].apply(mvoid_to_bson_id)
df_capsule["author user id"]=df_capsule["author user id"].apply(mvoid_to_bson_id)
#df["date updated"]=df["date updated"].astype('int').astype("datetime64[ms]")
#df["date created"]=df["date created"].astype('int').astype("datetime64[ms]")
df_capsule["is draft"]=df_capsule["is draft"].apply(lambda x: 1 if x==True else 0)
df_capsule["is public"]=df_capsule["is public"].apply(lambda x: 1 if x==True else 0)
df_capsule["approximate capsule duration"]=df_capsule["approximate capsule duration"].astype('float')

# Export de la base de capsule

In [ ]:
df_capsule.to_csv("BaseCapsule")

# Base de Cartes

In [159]:
from pandas import DataFrame
columns_map = ["Map id",
               "Node"
              ]
unwind_nodes = {"$unwind": "$graph.nodes"}
unwind_edges = {"$unwind": "$graph.edges"}
arrays_map = mon.aggregate(
"educanti", # database name
"map", # collection name
[
        unwind_nodes,
        unwind_edges
    ], # query spec
["_id",
 "graph.nodes.object",
 "name",
 "graph.edges.origin",
 "graph.edges.destination"
], # field names (in Mongo record)
[
        "id",
        "id",
        "string:100",
        "id",
        "id"
    ] # Monary field types (see below)
)
list_map = [{
        "Map Name": arrays_map[2][i],
        "map id": mvoid_to_bson_id(arrays_map[0][i]),
        "Capsule id": mvoid_to_bson_id(arrays_map[1][i]),
        "Origin": mvoid_to_bson_id(arrays_map[3][i]),
        "Destination": mvoid_to_bson_id(arrays_map[4][i])}
            
            for i in range(len(arrays_map[0]))]
df_map = DataFrame.from_dict(list_map)
#TO DO Récupérer uniquement les capsules qui correspondent à des origines ou des destinations

In [52]:
from pandas import DataFrame
columns_map_clean = ["Map id",
               "Node"
              ]
unwind_nodes = {"$unwind": "$graph.nodes"}
arrays_map_clean = mon.aggregate(
"educanti", # database name
"map", # collection name
[
        unwind_nodes
    ], # query spec
["_id",
 "graph.nodes.object",
 "name",
], # field names (in Mongo record)
[
        "id",
        "id",
        "string:100",
    ] # Monary field types (see below)
)
list_map_clean = [{
        "Map Name": arrays_map_clean[2][i],
        "map id": mvoid_to_bson_id(arrays_map_clean[0][i]),
        "Capsule id": mvoid_to_bson_id(arrays_map_clean[1][i])}
            
            for i in range(len(arrays_map_clean[0]))]
df_map_clean = DataFrame.from_dict(list_map_clean)
#TO DO Récupérer uniquement les capsules qui correspondent à des origines ou des destinations

In [53]:
df_map_clean[:20]

,Capsule id,Map Name,map id
0,56f3f55c0a728f32198b421a,Le guide des educantistes,56f407160a728f46042a2e9a
1,56f41b060a728f4dc50623e0,Les différentes parties d'une dissertation,56f420f20a728f4fdae09200
2,56f41b100a728f4dc4cb2ea0,Les différentes parties d'une dissertation,56f420f20a728f4fdae09200
3,56f41b190a728f4dc50623e1,Les différentes parties d'une dissertation,56f420f20a728f4fdae09200
4,56f5212e0a728f4fdb22d472,Les différentes parties d'une dissertation,56f420f20a728f4fdae09200
5,564f6fc10a728f0a68b6b085,Causality and policy evaluation,566023590a728f0bbfcadee9
6,56602a2d0a728f0da5f69aa3,Causality and policy evaluation,566023590a728f0bbfcadee9
7,56602a470a728f0da5f69aa4,Causality and policy evaluation,566023590a728f0bbfcadee9
8,56602a600a728f0da5f69aa5,Causality and policy evaluation,566023590a728f0bbfcadee9
9,56602a7f0a728f0da680bb93,Causality and policy evaluation,566023590a728f0bbfcadee9


In [131]:
df_test = pd.DataFrame.from_dict({'nodes':['C1','C1','C2','C2','C3','C3'],'Origin':['C1','C1','C1','C1','C1','C1'],'Destination':['C2','C3','C2','C3','C2','C3']})
a_test = df_test[df_test["nodes"] == df_test["Destination"]].groupby('Destination')['Origin'].apply(list)
u_test = pd.DataFrame(a_test).reset_index()
u_test.columns = ['Destination', 'list Origin']
df_test2 = pd.merge(left=u_test, right=df_test, on='Destination', how='inner')
df_test2.drop_duplicates(['Destination'],inplace=True)
df_test2

,Destination,list Origin,Origin,nodes
0,C2,[C1],C1,C1
3,C3,[C1],C1,C1


In [160]:
a = df_map[df_map["Capsule id"] == df_map["Origin"]].groupby('Origin')['Destination'].apply(list)
u = pd.DataFrame(a).reset_index()
u.columns = ['Origin', 'list Destination']
df_map2 = pd.merge(left=u, right=df_map, on='Origin', how = 'inner')
df_map2.drop_duplicates(['Origin'],inplace=True)
df_map2 = df_map2[['Origin', 'list Destination']]
df_map2=df_map2.reset_index(drop=True)
df_map2.head()

,Origin,list Destination
0,564d1d7f0a728f682a81f781,"[564d90020a728f682a82002f, 564d1d8b0a728f682a8..."
1,564d1d8b0a728f682a81f7e8,[564d1da40a728f682a81f8a4]
2,564d8f200a728f682a81fcfb,[564d1d980a728f682a81f846]
3,564d90020a728f682a82002f,[564d8f200a728f682a81fcfb]
4,564dc3e30a728f7559e667dd,"[564dc3f50a728f755ac63cc6, 564dc3f50a728f755ac..."


In [64]:
a = df_map[df_map["Capsule id"] == df_map["Destination"]].groupby('Destination')['Destination'].apply(list)
u = pd.DataFrame(a).reset_index()
u.columns = ['Destination', 'list Destination']
df_map3 = pd.merge(left=u, right=df_map, on='Origin', how = 'inner')
df_map3.drop_duplicates(['Origin'],inplace=True)
df_map3 = df_map3[['Origin', 'list Destination']]
df_map3=df_map3.reset_index(drop=True)
df_map3.head()

,Origin,list Destination
0,564d1d7f0a728f682a81f781,"[564d90020a728f682a82002f, 564d1d8b0a728f682a8..."
1,564d1d8b0a728f682a81f7e8,[564d1da40a728f682a81f8a4]
2,564d8f200a728f682a81fcfb,[564d1d980a728f682a81f846]
3,564d90020a728f682a82002f,[564d8f200a728f682a81fcfb]
4,564dc3e30a728f7559e667dd,"[564dc3f50a728f755ac63cc6, 564dc3f50a728f755ac..."


In [135]:
df_map2

,Destination,list Origin
0,564d1d8b0a728f682a81f7e8,[564d1d7f0a728f682a81f781]
1,564d1d980a728f682a81f846,[564d8f200a728f682a81fcfb]
2,564d1da40a728f682a81f8a4,[564d1d8b0a728f682a81f7e8]
3,564d8f200a728f682a81fcfb,[564d90020a728f682a82002f]
4,564d90020a728f682a82002f,[564d1d7f0a728f682a81f781]
5,564dc3f50a728f755ac63cc6,"[564dc3e30a728f7559e667dd, 564dc3ea0a728f755ac..."
6,564dc4130a728f755ac63cc7,"[564dc3f50a728f755ac63cc6, 564dc3f50a728f755ac..."
7,564dc49f0a728f7559e667e0,"[564dc3f50a728f755ac63cc6, 564dc3f50a728f755ac..."
8,564dc4b20a728f755ac63cc9,"[564dc49f0a728f7559e667e0, 564dc49f0a728f7559e..."
9,564f51b20a728f02c45d5794,"[564f5f760a728f02c30bb52c, 564f5f760a728f02c30..."


# Export de la Base Carte

In [137]:
df_map_clean.to_csv("BaseCarte")
df_map2.to_pickle("BaseLiens")

# Base Utilisateur

In [ ]:
columns_user = ["user id", "first name", "last name", "date joined"]
arrays_user = mon.query(
"educanti",
"user",
{},
["_id","first_name","last_name","date_joined"],
["id","string:30","string:30","date"],

)
df_user = np.matrix(arrays_user).transpose()
df_user = pd.DataFrame(df_user, columns=columns_user)
df_user["user id"] = df_user["user id"].apply(mvoid_to_bson_id)
df_user["user name"] = df_user["first name"] + " " + df_user["last name"]
df_user["date joined"] = df_user["date joined"].astype("int")
df_user = df_user[["user id","user name", "date joined"]]
df_user

# Export de la base utilisateur 

In [ ]:
df_user.to_csv("BaseUtilisateurs")

# Base de Test Attempts

In [ ]:
columns_test_attempt = [
    "Test Attempt id",
    #"date created",
    #"date updated",
    "Capsule id",
    "user id",
    "is completed",
    "is succeeded",
    "score"

]
arrays_test_attempt = mon.query(
"educanti", # database name
"test_attempt", # collection name
{}, # query spec
[
        "_id",
        #"date_created",
        #"date_updated",
        "test_container._ref.$id",
        "user",
        "is_completed",
        "is_succeeded",
        "score"

], # field names (in Mongo record)
[
        "id",
        #"date",
        #"date",
        "id",
        "id",
        "bool",
        "bool",
        "float32"
  
    ] # Monary field types (see below)
)
df_test_attempt = np.matrix(arrays_test_attempt).transpose() 

df_test_attempt = pd.DataFrame(df_test_attempt, columns=columns_test_attempt)
df_test_attempt["Test Attempt id"] = df_test_attempt["Test Attempt id"].apply(mvoid_to_bson_id)
df_test_attempt["user id"]=df_test_attempt["user id"].apply(mvoid_to_bson_id)
df_test_attempt["Capsule id"]=df_test_attempt["Capsule id"].apply(mvoid_to_bson_id)
#df_test_attempt["date updated"]=df_test_attempt["date updated"].astype('int').astype("datetime64[ms]")
#df_test_attempt["date created"]=df_test_attempt["date created"].astype('int').astype("datetime64[ms]")
df_test_attempt

# Sauvegarde de la base de Test Attempts

In [ ]:
df_test_attempt.to_csv("BaseTestAttempts")

# Let's Play with our Statements

# Question DataSet

In [ ]:
def get_question_number(question):
    l = question.strip(']').split('[')
    n = len(l)
    return l[-1]

In [ ]:
columns_questions = [
    "Capsule id",
    "date created",
    "date updated",
    "user id",
    "test attempt",
    "question number",
    "duration",
    "success"

]
arrays_questions = mon.query(
"educanti", # database name
"statement", # collection name
{"verb.identifier": "http://adlnet.gov/expapi/verbs/answered"}, # query spec
[
        "context.contextActivities.parent.0.object._ref.$id",
        "date_created",
        "date_updated",
        "actor.account.user",
        "object.definition.object._ref.$id",
        "object.definition.sub_element",
        "result.duration",
        "result.success"

], # field names (in Mongo record)
[
        "id",
        "date",
        "date",
        "id",
        "id",
        "string:100",
        "float32",
        "bool"
  
    ] # Monary field types (see below)
)
df_questions = np.matrix(arrays_questions).transpose() 

df_questions = pd.DataFrame(df_questions, columns=columns_questions)

In [ ]:
df_questions["Capsule id"] = df_questions["Capsule id"].apply(mvoid_to_bson_id).astype("string")
df_questions["user id"]=df_questions["user id"].apply(mvoid_to_bson_id).astype("string")
df_questions["test attempt"]=df_questions["test attempt"].apply(mvoid_to_bson_id).astype("string")
df_questions["date updated"]=df_questions["date updated"].astype('int')
df_questions["date created"]=df_questions["date created"].astype('int')
df_questions["question number"]=df_questions["question number"].apply(get_question_number)
df_questions["success"]=df_questions["success"].apply(lambda x: 1 if x==True else 0)
df_questions["duration"]=df_questions["duration"].astype('float')

# Export Base Questions

In [ ]:
df_questions.to_csv("BaseQuestions")

# Amélioration de la base

## Import des bases de données


In [ ]:
df_capsule = pd.read_csv("BaseCapsule", index_col=0)
df_user = pd.read_csv("BaseUtilisateurs",index_col=0)
df_test_attempt = pd.read_csv("BaseTestAttempts",index_col=0)
df_questions = pd.read_csv("BaseQuestions",index_col=0)
df_map = pd.read_csv("BaseCarte",index_col=0)
df_liens = pd.read_pickle("BaseLiens")
df_liens

In [164]:
df_map2 = df_map2.rename(columns={'Origin':'Capsule id'})
df_map2.head()

,Capsule id,list Destination
0,564d1d7f0a728f682a81f781,"[564d90020a728f682a82002f, 564d1d8b0a728f682a8..."
1,564d1d8b0a728f682a81f7e8,[564d1da40a728f682a81f8a4]
2,564d8f200a728f682a81fcfb,[564d1d980a728f682a81f846]
3,564d90020a728f682a82002f,[564d8f200a728f682a81fcfb]
4,564dc3e30a728f7559e667dd,"[564dc3f50a728f755ac63cc6, 564dc3f50a728f755ac..."


In [71]:
df_test_attempt = pd.read_csv("BaseTestAttempts",index_col=0)
df_test_attempt = df_test_attempt.rename(columns={'Test Attempt id':'test attempt'})[['test attempt','is completed','is succeeded','score']]

In [72]:
df_questions_user = pd.merge(left=df_user,right=df_questions,on="user id",how ='inner')

In [73]:
df_questions_user_capsule = pd.merge(left=df_capsule,right=df_questions_user,on="Capsule id",how ='inner')

In [74]:
df_user = df_user.rename(columns={'user id': 'author user id','user name':'author user name'})

In [75]:
df_questions_user_author_capsule = pd.merge(left=df_user[["author user id","author user name"]],right=df_questions_user_capsule,on="author user id",how ='inner')

In [76]:
df_questions_user_author_capsule_test = pd.merge(left=df_test_attempt,right=df_questions_user_author_capsule,on="test attempt",how='inner')
df_questions_user_author_capsule_test["is completed"]=df_questions_user_author_capsule_test["is completed"].apply(lambda x: 1 if x==True else 0)
df_questions_user_author_capsule_test["is succeeded"]=df_questions_user_author_capsule_test["is succeeded"].apply(lambda x: 1 if x==True else 0)

In [77]:
df_questions_user_author_capsule_test["is user author"] = (df_questions_user_author_capsule_test["author user id"] == df_questions_user_author_capsule_test["user id"]).apply(lambda x: 1 if x==True else 0)

In [78]:
df_all_map = pd.merge(left=df_questions_user_author_capsule_test,right=df_map,on="Capsule id",how = 'left')

In [84]:
df_all_map = df_all_map.dropna()

In [165]:
df = pd.merge(left=df_map2, right=df_all_map,on='Capsule id', how = 'right')

In [166]:
df

,Capsule id,list Destination,test attempt,is completed,is succeeded,score,author user id,author user name,capsule name,type of author,...,user name,date joined,date created,date updated,question number,duration,success,is user author,Map Name,map id
0,56b371740a728f745f33a0b0,NaN,57288acd0a728f7346c5567c,1,1,1.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Audrey Chagneau,1454539430147,1467817861110,1467817861130,0,30.591999,1,1,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
1,56b371740a728f745f33a0b0,NaN,57288acd0a728f7346c5567c,1,1,1.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Audrey Chagneau,1454539430147,1467817861140,1467817861143,1,23.615999,1,1,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
2,56b371740a728f745f33a0b0,NaN,57288acd0a728f7346c5567c,1,1,1.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Audrey Chagneau,1454539430147,1467818354705,1467818354724,0,30.591999,1,1,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
3,56b371740a728f745f33a0b0,NaN,57288acd0a728f7346c5567c,1,1,1.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Audrey Chagneau,1454539430147,1467818354734,1467818354737,1,23.615999,1,1,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
4,56b371740a728f745f33a0b0,NaN,5728b7570a728f761d18fde5,1,0,0.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Anne-Sophie Jauneau,1461589047971,1467818583495,1467818583511,0,14.046000,0,0,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
5,56b371740a728f745f33a0b0,NaN,5728b7570a728f761d18fde5,1,0,0.000,56b282a60a728f56a0f9874f,Audrey Chagneau,Premier modèle de la tectonique des plaques de...,user,...,Anne-Sophie Jauneau,1461589047971,1467818583521,1467818583524,1,4.430000,0,0,Modèle de la tectonique des plaques,56b371110a728f745f33a09a
6,564dc3e30a728f7559e667dd,NaN,564dc8680a728f755ac642b9,1,0,1.000,56549b380a728f30e4f58d0c,Lucas Gruez,Citoyenneté et nationalité,user,...,Arnaud Riegert,1448059402000,1467817860588,1467817860633,0,5.590000,1,0,Les institutions,56d5d4f80a728f3276060c68
7,564dc3e30a728f7559e667dd,NaN,564dc8680a728f755ac642b9,1,0,1.000,56549b380a728f30e4f58d0c,Lucas Gruez,Citoyenneté et nationalité,user,...,Arnaud Riegert,1448059402000,1467817860588,1467817860633,0,5.590000,1,0,Citoyenneté française et citoyenneté européenn...,5682adb50a728f2e9cad92a2
8,564dc3e30a728f7559e667dd,NaN,564dc8680a728f755ac642b9,1,0,1.000,56549b380a728f30e4f58d0c,Lucas Gruez,Citoyenneté et nationalité,user,...,Arnaud Riegert,1448059402000,1467817860644,1467817860650,1,12.302000,1,0,Les institutions,56d5d4f80a728f3276060c68
9,564dc3e30a728f7559e667dd,NaN,564dc8680a728f755ac642b9,1,0,1.000,56549b380a728f30e4f58d0c,Lucas Gruez,Citoyenneté et nationalité,user,...,Arnaud Riegert,1448059402000,1467817860644,1467817860650,1,12.302000,1,0,Citoyenneté française et citoyenneté européenn...,5682adb50a728f2e9cad92a2


In [87]:
df.to_csv("BaseFinale")

# Commencer ici

In [90]:
df = pd.read_csv("BaseFinale",index_col=0)
base_continent = df[df["Map Name"] == "Naissance d'une théorie:  de la dérive des continents à l'expansion océanique"]

In [ ]:
pd.value_counts(df["Map Name"])

# Base de travail

In [ ]:
new_columns = [
    "is completed",
    "is succeeded",
    "score",
    #"type of author",
    "is draft",
    "is public",
    'approximate capsule duration',
    "date joined",
    "date created",
    "date updated",
    "question number",
    "duration",
    "success",
    "is user author"
]
df = df[new_columns]
df.head()

# Traitement Base de Données

In [ ]:
numerique = [ c for c,d in zip(df.columns,df.dtypes) if d == np.int64 and c not in ["success"] ]
categories = [ c for c in df.columns if c not in numerique and c not in ["success"] ]
target = "success"
num = df[ numerique ]
cat = df[ categories ]
tar = df[ target ]
prep = DictVectorizer()
cat_as_dicts = [dict(r.iteritems()) for _, r in cat.iterrows()]
temp = prep.fit_transform(cat_as_dicts)
cat_exp = temp.toarray()
cat_exp_df = pd.DataFrame( cat_exp, columns = prep.feature_names_ )
X = (pd.concat ( [ num, cat_exp_df ], axis= 1))
Y = (tar)

# Machine Learning


In [ ]:
from sklearn.cross_validation import train_test_split, cross_val_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state = 0)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier


clf = Pipeline([
      ('normalize', Normalizer()),
      ('classifier', GradientBoostingClassifier(random_state = 1))
    ])
clf2 = XGBClassifier(missing=np.nan,gamma = 1.5, max_depth=50, n_estimators=350, learning_rate=0.01, nthread=30, subsample=0.95, colsample_bytree=0.90, seed=4242)
clf2 = clf2.fit(X_train, Y_train.ravel())

In [ ]:
from sklearn.metrics import confusion_matrix
for x,y in [ (X_train, Y_train), (X_test, Y_test) ]:
    yp  = clf2.predict(x)
    cm = confusion_matrix(y.ravel(), yp.ravel())
    print(cm)
import matplotlib.pyplot as plt
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
savefig("conf_mat1.png")

In [ ]:
cm

In [ ]:
from sklearn import metrics
metrics.classification_report(Y_test,clf2.predict(X_test))

In [ ]:
from sklearn.metrics import roc_curve, auc
probas = clf2.predict_proba(X_test)
probas[:5]

In [ ]:
rep = [ ]
yt = Y_test.ravel()
for i in range(probas.shape[0]):
    p0,p1 = probas[i,:]
    exp = yt[i]
    if p0 > p1 :
        if exp == 0 :
            # bonne réponse
            rep.append ( (1, p0) )
        else :
            # mauvaise réponse
            rep.append( (0,p0) )
    else :
        if exp == 0 :
            # mauvaise réponse
            rep.append ( (0, p1) )
        else :
            # bonne réponse
            rep.append( (1,p1) )
mat_rep = numpy.array(rep)
mat_rep[:5]
mat_rep

In [ ]:
"taux de bonne réponse",sum(mat_rep[:,0]/len(mat_rep)) # voir matrice de confusion

In [ ]:
fpr, tpr, thresholds = roc_curve(mat_rep[:,0], mat_rep[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
savefig("ROC.png")

# Bayes Network MotherF####R

In [128]:
liste_nom = list(set([(a,b) for a,b in (base_continent[["capsule name","Capsule id"]].values)]))
list_orig_dest = list(set([(a,b) for a,b in (base_continent[["capsule name","list Destination"]].values)]))
list_orig_dest

[('Les arguments pour et contre la mobilit\xc3\xa9 des continents de Wegener',
  "[u'56b36f040a728f745f33a070', u'56b36f470a728f745f33a075']"),
 ('Flux g\xc3\xa9othermique au niveau de la dorsale',
  "[u'56b36fbe0a728f745f33a085']"),
 ('Plan de Wadatti-Benioff ', "[u'56b370370a728f745f33a08b']"),
 ("Th\xc3\xa9orie de l'expansion oc\xc3\xa9anique",
  "[u'56b370370a728f745f33a08b']"),
 ('Contraste Oc\xc3\xa9an-Continents', "[u'56b36fbe0a728f745f33a085']"),
 ('Anomalies magn\xc3\xa9tiques ', "[u'56b36fbe0a728f745f33a085']"),
 ('Utilisation des ondes sismiques', "[u'56b36f700a728f745f33a079']"),
 ('Mod\xc3\xa8le de la subduction', nan),
 ('Roches et min\xc3\xa9raux 1S', "[u'56b36f700a728f745f33a079']")]

In [129]:
liste_nom

[('Mod\xc3\xa8le de la subduction', '56b370370a728f745f33a08b'),
 ('Roches et min\xc3\xa9raux 1S', '56b36f040a728f745f33a070'),
 ('Flux g\xc3\xa9othermique au niveau de la dorsale',
  '56b36fa50a728f745f33a082'),
 ('Les arguments pour et contre la mobilit\xc3\xa9 des continents de Wegener',
  '56b36eb10a728f745f33a066'),
 ('Utilisation des ondes sismiques', '56b36f470a728f745f33a075'),
 ('Plan de Wadatti-Benioff ', '56b370270a728f745e878e2e'),
 ("Th\xc3\xa9orie de l'expansion oc\xc3\xa9anique",
  '56b36fbe0a728f745f33a085'),
 ('Contraste Oc\xc3\xa9an-Continents', '56b36f700a728f745f33a079'),
 ('Anomalies magn\xc3\xa9tiques ', '56b36fde0a728f745e878e28')]

In [126]:
base_continent["list Destination"].values

array(["[u'56b36f040a728f745f33a070', u'56b36f470a728f745f33a075']",
       "[u'56b36f040a728f745f33a070', u'56b36f470a728f745f33a075']",
       "[u'56b36f040a728f745f33a070', u'56b36f470a728f745f33a075']", ...,
       nan, nan, nan], dtype=object)